GETTING THE DATA

In [2]:
from elasticsearch import Elasticsearch
from tqdm import tqdm
import requests as r

Question 1 Running Elastic

In [7]:
es = Elasticsearch('http://localhost:9200')

In [10]:
print('ans:', es.info()['version']['build_hash'])

ans: 42f05b9372a9a4a470db3b52817899b99a76ee73


In [11]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [13]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

Question 2

In [14]:
index_settings = dict(
    settings=dict(number_of_shards=1, number_of_replicas=0),
    mappings=dict(
        properties=dict(
            text=dict(type='text'),
            section=dict(type='text'),
            question=dict(type='text'),
            course=dict(type='keyword')
        )
    )
)
index_name = 'course-questions'

In [16]:
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████| 948/948 [00:21<00:00, 44.21it/s]


Question 3

In [17]:
query = 'How do I execute a command in a running docker container?'
query_template = dict(
    size=5,
    query=dict(
        bool=dict(
            must=dict(multi_match=dict(query=query, fields=['question^4', 'text'], type='best_fields')),
        )
    )
)

In [18]:
response = es.search(index=index_name, body=query_template)
best_score = response['hits']['hits'][0]['_score']

print('Answer:', best_score)

Answer: 84.050095


Q4 Filtering

In [26]:
query = 'How do I execute a command in a running docker container?'
query_template = dict(
    size=5,  # Increase the size to get more hits
    query=dict(
        bool=dict(
            must=dict(multi_match=dict(query=query, fields=['question^4', 'text'], type='best_fields'))
            ,filter=dict(term=dict(course='machine-learning-zoomcamp'))
        )
    )
)

response = es.search(index=index_name, body=query_template)
hits = [m['_source'] for m in response['hits']['hits']]

# Check if there are enough hits
if len(hits) >= 3:
    third_question = hits[2]['question']
    print('Answer:', third_question)
else:
    print('Not enough hits found to extract the third question. Only found {} hits.'.format(len(hits)))


Not enough hits found to extract the third question. Only found 0 hits.


In [ ]:
Q